In [1]:
# Jan 30, 2020
# Joined the 5 classification tables available
# to be evaluated:
    # join is left by default
    # because I adopt cit_received as the leftmost data
        # this dataset keep data on the patents who received citation
    # I can make the first join as an 'outer' operation between cit_received and cit_made
    # To be complete, I can begin with patent.csv and make the first two joins with 'outer' type
#     Ill git add now and try this
    

# Jan 29, 2020
# many errors appeared after joining the dataset 
# i rewritten code into simpler scripts
# this script joins variables together to be processed later

# they are:
#     - cit_delay - reads clean_patent and clean_uspatentcitation and calculates delay by patent
#     - cit_made - reads clean_uspatentcitation and calculates citations made 
#     - cit_received - reads clean_uspatentcitation and calculates citations received
#     - cit_tree - reads clean_uspatentcitation and cit_made and calculates parent_citation
#     - generalit - reads wipo and clean_uspatentcitation and calculates generality
#     - originality - reads wipo and clean_uspatentcitation and calculates originality
#     - wipo_first_class - reads wipo and generates wipo_first_class

# it is now way faster, but I cannot know for sure if this is a consequence of coding or something changed in 
# infrastructure. 

# since class is a variable that also appear in non-cited patents, I'll include them in the final step


# Jan, 20th 2020
# join delay and tree data to form the analysis dataset


In [2]:
import pandas as pd
import numpy as np

In [3]:
def data_read(file, names, usecols, dtype):
    df = pd.read_csv(file, names=names, usecols=usecols, dtype=dtype, index_col='id', header=0)
    dfs.append(df)

In [4]:
patent='data/cleanpatent.csv'

received='data/cit_received.csv'
made='data/cit_made.csv'
received_delay='data/cit_received_delay.csv'
made_delay='data/cit_made_delay.csv'
cit_tree = 'data/cit_tree.csv'
originality = 'data/originality.csv'
generality = 'data/generality.csv'

dst='data/dataset.csv'

wipo = 'data/wipo.csv'
ipcr = 'data/ipcr.csv'
cpc = 'data/cpc.csv'
nber = 'data/nber.csv'
uspc = 'data/uspc.csv'

centrality='data/eigen.csv'
eigen='data/eigen2.csv'


In [5]:
dfs=[]

In [6]:
#adopting patent.csv as the reference of patents
df = pd.read_csv(patent, dtype=object)
df['num_claims']=df['num_claims'].astype(float) #int does not handle NAN values, so using float instead
df.set_index('id', inplace=True)

In [7]:
#outer join to citation received
#there are patents who received citations that are not in patent.csv
#for example, the very ancient ones

names=['index', 'id', 'cit_received']
usecols=['id', 'cit_received']
dtype={'id':object, 'cit_received':float}
df2 = pd.read_csv(received, names=names, usecols=usecols, dtype=dtype, header=0)
df2.set_index('id', inplace=True)

df=df.join(df2, how='outer')
df['cit_received']=df['cit_received'].fillna(0)
dfs.append(df)
df2=[]

In [8]:
#i dont see any reason for patents making citations not to appear in patent.csv
#so from this data on, the list of patents is stable
#the number of rows should not change (check)

names=['index', 'id', 'cit_made']
usecols=['id', 'cit_made']
dtype={'id':object}
data_read(made, names, usecols, dtype)

In [9]:
names=['id', 'cit_received_delay']
dtype={'id':object}
usecols=['id', 'cit_received_delay']
data_read(received_delay, names, usecols, dtype)

In [10]:
names=['id', 'cit_made_delay']
dtype={'id':object}
usecols=['id', 'cit_made_delay']
data_read(made_delay, names, usecols, dtype)

In [11]:
names=['id', 'parent_citation']
dtype={'id':object}
usecols=['id', 'parent_citation']
data_read(cit_tree, names, usecols, dtype)

In [12]:
names=['id', 'df_squared','total_citation','herfindal','originality']
dtype={'id':object}
usecols=['id', 'originality']
data_read(originality, names, usecols, dtype)

In [13]:
names=['id', 'df_squared','total_citation','herfindal','generality']
dtype={'id':object}
usecols=['id', 'generality']
data_read(generality, names, usecols, dtype)

In [14]:
df = pd.read_csv(wipo, dtype=object)
df.set_index('id', inplace=True)
dfs.append(df)

In [15]:
df = pd.read_csv(ipcr, dtype=object)
df.set_index('id', inplace=True)
dfs.append(df)

In [16]:
df = pd.read_csv(cpc, dtype=object)
df.set_index('id', inplace=True)
dfs.append(df)

In [17]:
df = pd.read_csv(nber, dtype=object)
df.set_index('id', inplace=True)
dfs.append(df)

In [18]:
df = pd.read_csv(uspc, dtype=object)
df.set_index('id', inplace=True)
dfs.append(df)

In [19]:
# centrality
df = pd.read_csv(centrality, dtype=object, usecols=['id','katz', 'pagerank'])
df.set_index('id', inplace=True)
dfs.append(df)

In [20]:
# eigen
df = pd.read_csv(eigen, dtype=object)
df.set_index('id', inplace=True)
dfs.append(df)

In [21]:
# dfs = [df, df_made, df_received_delay]
# dfs = [df.set_index('id') for df in dfs]
df=dfs[0].join(dfs[1:])

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8702317 entries, 0000000 to re25727
Data columns (total 26 columns):
type                   object
date                   object
kind                   object
num_claims             float64
cit_received           float64
cit_made               float64
cit_received_delay     float64
cit_made_delay         float64
parent_citation        float64
originality            float64
generality             float64
wipo_sector_id         object
wipo_field_id          object
ipcr_section           object
ipcr_ipc_class         object
ipcr_subclass          object
cpc_section_id         object
cpc_subsection_id      object
cpc_group_id           object
nber_category_id       object
nber_subcategory_id    object
uspc_mainclass_id      object
uspc_subclass_id       object
pagerank               object
katz                   object
eigen                  object
dtypes: float64(8), object(18)
memory usage: 1.8+ GB


In [23]:
df.sample(n=10).transpose()

id,3877735,9034909,6415899,2974710,3993335,3548294,2697936,4194939,D313358,4662728
type,NaN,utility,utility,NaN,utility,NaN,NaN,utility,design,utility
date,NaN,2015-05-19,2002-07-09,NaN,1976-11-23,NaN,NaN,1980-03-25,1991-01-01,1987-05-05
kind,NaN,B2,B1,NaN,A,NaN,NaN,A,S,A
num_claims,NaN,6,12,NaN,13,NaN,NaN,5,1,8
cit_received,5,0,3,3,18,4,7,2,0,10
cit_made,NaN,NaN,10,NaN,5,NaN,NaN,5,4,7
cit_received_delay,26.531,NaN,7.89772,34.5753,20.2729,24.9507,32.6438,6.36575,NaN,12.1411
cit_made_delay,NaN,NaN,20.5756,NaN,21.7781,NaN,NaN,19.2268,54.7692,15.4114
parent_citation,45,NaN,5,19,165,88,88,75,NaN,375
originality,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0.5


In [24]:
df.describe()

/home/applications/anaconda3/4.2.0/lib/python3.5/site-packages/numpy/lib/function_base.py:4269: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)


,num_claims,cit_received,cit_made,cit_received_delay,cit_made_delay,parent_citation,originality,generality
count,6.488254e+06,8.702317e+06,6.183715e+06,7.046552e+06,6.175204e+06,6.256483e+06,5.604693e+06,6.552762e+06
mean,1.414322e+01,1.050814e+01,1.477889e+01,2.044491e+01,1.367688e+01,2.472962e+02,1.497649e-01,1.471856e-01
std,1.200423e+01,2.794500e+01,4.194739e+01,2.435857e+01,1.069101e+01,1.588333e+03,2.102631e-01,2.107271e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,-2.858301e+02,-2.805233e+02,1.000000e+00,0.000000e+00,0.000000e+00
25%,NaN,1.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,3.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,1.000000e+01,NaN,NaN,NaN,NaN,NaN,NaN
max,8.870000e+02,3.922000e+03,5.811000e+03,2.075753e+02,1.827425e+02,2.274790e+05,8.000000e-01,8.000000e-01


In [25]:
df.describe(include=[np.object]).transpose()

,count,unique,top,freq
type,6488253,7,utility,5879882
date,6488256,2218,2017-04-18,7602
kind,6488253,15,B2,2920417
wipo_sector_id,5879781,5,1,2158667
wipo_field_id,5879781,35,06,528686
ipcr_section,5937053,8,G,1492884
ipcr_ipc_class,5937053,137,1,599183
ipcr_subclass,5937053,294,B,1061448
cpc_section_id,5860545,8,G,1453783
cpc_subsection_id,5860545,122,H04,568315


In [26]:
# df=df.merge(wipo, left_index=True, right_index=True, how='outer')

In [27]:
# df.sample(n=10)

In [28]:
# df.describe()

In [29]:
df.to_csv(dst)

In [30]:
# this code includes WIPO in the dataset
# as class is also part of non-cited patents, I'm not including in here.
# in any case, for some reason this merge could not be done by the method used in the previous
# there is a bug when using read_csv using dtype and indexing. 
# it is a known bug in the community but i do not know why the previous join did not accused the issue
# the method below corrects the issue 

# names=['index', 'id', 'field_id']
# dtype={'id':object}
# usecols=['id', 'field_id']
# # data_read(wipo, names, usecols, dtype)
# wipo = pd.read_csv(wipo, names=names, usecols=usecols, dtype=dtype, header=0)
# wipo['id']=wipo['id'].astype(str)
# wipo=wipo.set_index('id')
# wipo.info()

# dfs.append(wipo)